In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from s3 import S3
from depth_focus import apply_depth_based_gaussian
from cpbd import compute_cpbd

In [ ]:
# Load an RGBD image
rgb_image = cv.imread("rgb.jpg")
depth_image = cv.imread("depth.png", cv.IMREAD_GRAYSCALE)
# Load anoter RGB image
image = cv.imread("tid2008/reference_images/I01.BMP")

In [ ]:
# Apply different types of blurs
blurred_gaussian_images = [image]
for sigma in [0.5, 1.0, 2.0]:
    blurred_image = cv.GaussianBlur(image, (0, 0), sigma)
    blurred_gaussian_images.append(blurred_image)
# Depth blur using the depth image
blurred_depth_images = [rgb_image]
for focused_depth_val in [50, 100, 150]:
    depth_blurred_image = apply_depth_based_gaussian(rgb_image, depth_image, focused_depth_val)
    blurred_depth_images.append(depth_blurred_image)

# Add motion blur
blurred_motion_images = []

In [ ]:
# Compute blur level estimations using CPBD
cpbd_results_blurred_gaussian_images = []
for img in blurred_gaussian_images:
    result = compute_cpbd(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    cpbd_results_blurred_gaussian_images.append(result) # A single scalar value

cpbd_results_depth_images = []
for img in blurred_depth_images:
    result = compute_cpbd(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    cpbd_results_depth_images.append(result) # A single scalar value

cpbd_results_blurred_motion_images = []
for img in blurred_motion_images:
    result = compute_cpbd(cv.cvtColor(img, cv.cv.COLOR_BGR2GRAY))
    cpbd_results_blurred_motion_images.append(result) # A single scalar value

In [ ]:
# Compute blur level estimations using S3
s3 = S3()
s3_results_blurred_gaussian_images = []
for img in blurred_gaussian_images:
    result = s3.compute_s3(img) # s1, s2, s3 maps and the final score
    s3_results_blurred_gaussian_images.append(result)

s3_results_blurred_depth_images = []
for img in blurred_depth_images:
    result = s3.compute_s3(img) # s1, s2, s3 maps and the final score
    s3_results_blurred_depth_images.append(result)

s3_results_blurred_motion_images = []
for img in blurred_motion_images:
    result = s3.compute_s3(img) # s1, s2, s3 maps and the final score
    s3_results_blurred_motion_images.append(result)

In [ ]:
# Save results as images
def normalize_float_image_for_saving(img, cmap='gray'):
    norm_img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX, dtype=cv.CV_32F)
    return norm_img

results_folder = "results/"
os.makedirs(results_folder, exist_ok=True)
for i, img in enumerate(blurred_gaussian_images):
    s3_score = s3_results_blurred_gaussian_images[i][3]  # Final score
    cv.imwrite(results_folder + f"blurred_gaussian_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_blurred_gaussian_{i}.png", normalize_float_image_for_saving(s3_results_blurred_gaussian_images[i][0]))
    cv.imwrite(results_folder + f"s2_blurred_gaussian_{i}.png", normalize_float_image_for_saving(s3_results_blurred_gaussian_images[i][1]))
    cv.imwrite(results_folder + f"s3_blurred_gaussian_{i}.png", normalize_float_image_for_saving(s3_results_blurred_gaussian_images[i][2]))

for i, img in enumerate(blurred_depth_images):
    s3_score = s3_results_blurred_depth_images[i][3]  # Final score
    cv.imwrite(results_folder + f"blurred_depth_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_blurred_depth_{i}.png", normalize_float_image_for_saving(s3_results_blurred_depth_images[i][0]))
    cv.imwrite(results_folder + f"s2_blurred_depth_{i}.png", normalize_float_image_for_saving(s3_results_blurred_depth_images[i][1]))
    cv.imwrite(results_folder + f"s3_blurred_depth_{i}.png", normalize_float_image_for_saving(s3_results_blurred_depth_images[i][2]))

for i, img in enumerate(blurred_motion_images):
    s3_score = s3_results_blurred_motion_images[i][3]  # Final score
    cv.imwrite(results_folder + f"blurred_motion_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_blurred_motion_{i}.png", normalize_float_image_for_saving(s3_results_blurred_motion_images[i][0]))
    cv.imwrite(results_folder + f"s2_blurred_motion_{i}.png", normalize_float_image_for_saving(s3_results_blurred_motion_images[i][1]))
    cv.imwrite(results_folder + f"s3_blurred_motion_{i}.png", normalize_float_image_for_saving(s3_results_blurred_motion_images[i][2]))


In [ ]:
# Load different images to show their blur maps using S3
folder = "tid2008/reference_images/"
images = [
    cv.imread(folder + "I07.BMP"),
    cv.imread(folder + "I17.BMP"),
    cv.imread(folder + "I23.BMP")]

# Compute S3 maps for these images
s3_results_images = []
for img in images:
    result = s3.compute_s3(img)  # s1, s2, s3 maps and the final score
    s3_results_images.append(result)

# Save the results
for i, img in enumerate(images):
    s3_score = s3_results_images[i][3]  # Final score
    cv.imwrite(results_folder + f"image_{i}_s3_{s3_score:.2f}.png", img)
    cv.imwrite(results_folder + f"s1_image_{i}.png", normalize_float_image_for_saving(s3_results_images[i][0]))
    cv.imwrite(results_folder + f"s2_image_{i}.png", normalize_float_image_for_saving(s3_results_images[i][1]))
    cv.imwrite(results_folder + f"s3_image_{i}.png", normalize_float_image_for_saving(s3_results_images[i][2]))